# Dune Migration


_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

***

## 1. Import Python Packages and Start GRASS Session

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("nags_head/PERMANENT");
# gj.init("nags_head/flooding");

In [ ]:
# Make a new mapset for this part
gs.run_command("g.mapset", mapset="dune_migration", location="nags_head", flags="c")

Before we get started, we check the computational region.

In [ ]:
!g.region region=jockeys_ridge -p

***

## 2. Getting Started with GRASS Temporal Tools

In [ ]:
gs.run_command('t.create', output='JockeysRidge', type='strds',
                temporaltype='relative', title="Jockeys Ridge Elevation Series",
                description="from 1996 to 2020 with gaps")

In [ ]:
dems = gs.read_command("g.list", type="vector", pattern="JR_????", separator="comma")
dems

In [ ]:
gs.run_command("t.register", maps=dems, input="JockeysRidge", type="raster", unit="years")

Check what you have now in JockeysRidge dataset.

In [ ]:
print(gs.read_command("t.rast.list", input="JockeysRidge"))

Set the same color table for all maps.

In [ ]:
gs.run_command("t.rast.colors", input="JockeysRidge", color="elevation")

Animate the time series.

In [ ]:
nh_animation = gj.TimeSeriesMap()
nh_animation.add_raster_series("JockeysRidge")
nh_animation.d_legend(color="black", at=(12,72,0,3), fontsize=12)
nh_animation.d_barscale(length=250)
# nh_animation.show()

In [ ]:
timemap.save(filename="flood.gif");

In [ ]:
from IPython.display import display, Image

display(Image(filename="flood.gif"))